# 1. Install and Import Dependencies

In [ ]:
'''!pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html'''

In [1]:
!pip install transformers requests beautifulsoup4 pandas numpy

     |████████████████████████████████| 2.3MB 24.0MB/s 
     |████████████████████████████████| 3.3MB 41.6MB/s 
     |████████████████████████████████| 901kB 36.9MB/s 


In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

# 2. Instantiate Model

In [3]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

# 3. Encode and Calculate Sentiment

In [4]:
tokens = tokenizer.encode('It was good but couldve been better. Great', return_tensors='pt')

In [5]:
result = model(tokens)

In [6]:
result.logits

tensor([[-2.7768, -1.2353,  1.4419,  1.9804,  0.4584]],
       grad_fn=<AddmmBackward>)

In [7]:
int(torch.argmax(result.logits))+1

4

# 4. Collect Reviews

In [8]:
r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [9]:
reviews

['Still one of the favorite coffee shop in Sydney. Staffs have excellent knowledge about beans , flavor, brew skills. They make own pastries which are also tasty too.',
 'Good coffee and toasts. Straight up and down - hits the spot with nothing mind blowing. Solid and tasty. \xa0Good work',
 "Ricotta hot cakes! These were so yummy. I ate them pretty fast and didn't share with anyone because they were that good ;). I ordered a green smoothie to balance it all out. Smoothie was a nice way to end my brekkie at this restaurant. Others with me ordered the salmon Benedict and the smoked salmon flatbread. They were all delicious and all plates were empty. Cheers!",
 "Ron & Jo are on the go down under and Wow! \xa0We've found our breakfast place in Sydney and are coming back before we leave! \xa0 I yelped you the Social Brew Cafe which was a few blocks from where we are staying so off we went. \xa0They have seating outside and inside and it was a little chilly and they had a table available in

# 5. Load Reviews into DataFrame and Score

In [10]:
import numpy as np
import pandas as pd

In [11]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [12]:
df['review'].iloc[0]

'Still one of the favorite coffee shop in Sydney. Staffs have excellent knowledge about beans , flavor, brew skills. They make own pastries which are also tasty too.'

In [13]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [15]:
sentiment_score(df['review'].iloc[1])

5

In [16]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [17]:
df

,review,sentiment
0,Still one of the favorite coffee shop in Sydne...,5
1,Good coffee and toasts. Straight up and down -...,5
2,Ricotta hot cakes! These were so yummy. I ate ...,5
3,Ron & Jo are on the go down under and Wow! We...,5
4,Great coffee and vibe. That's all you need. C...,5
5,Great coffee and vibe. That's all you need. C...,4
6,We came for brunch twice in our week-long visi...,4
7,This is my fave brunch café in and around Sydn...,5
8,Delicious. The waitress was hot. The burger wa...,4
9,Rainy day in Sydney. Borrowed an umbrella and ...,5


In [18]:
df['review'].iloc[3]

"Ron & Jo are on the go down under and Wow! \xa0We've found our breakfast place in Sydney and are coming back before we leave! \xa0 I yelped you the Social Brew Cafe which was a few blocks from where we are staying so off we went. \xa0They have seating outside and inside and it was a little chilly and they had a table available inside so we grabbed it. \xa0The menu has a lot of tantalizing choices and our very nice waitress made some suggestions that we took. \xa0Jo went with the ricotta pancakes with iced tea and I ordered the smoked salmon with a macchiato. \xa0Our drinks were served quickly and my wife's crimson iced tea had a nice flavor and macchiato was served dark and rich. \xa0Jo's pancakes came with a liberal number of fresh strawberries and the ricotta was spread between the first and second pancake with a pistachio butter (lots of ground pistachios) in between the second and third cake. \xa0These pancakes were well presented and so delicious! \xa0Jo order a side of bacon whi